# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [8]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/home/ubuntu/workspace/src/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/home/ubuntu/workspace/src/emails/ham', 'ham'))


/home/ubuntu/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Let's have a look at that DataFrame:

In [9]:
data.head()

,class,message
/home/ubuntu/workspace/src/emails/spam/00253.83b95b05e275286eddcf557ea581e754,spam,There is NO stumbling on to it! \n\n\n\nThe gr...
/home/ubuntu/workspace/src/emails/spam/00333.4bb36a535cb3d738f30f985f1e10a786,spam,------000000000000000000000\n\nContent-Type: t...
/home/ubuntu/workspace/src/emails/spam/00165.45db168e8e1a78a66972b9f50c47b6dc,spam,"<FONT SIZE=1 COLOR=""000000"">To be removed plea..."
/home/ubuntu/workspace/src/emails/spam/00116.29e39a0064e2714681726ac28ff3fdef,spam,HABERDAR.COM - HABER VE MEDYA PORTALI\n\nArtýk...
/home/ubuntu/workspace/src/emails/spam/00150.f97c73fa56460a6afc6d9418ad76b5b5,spam,This is a multi-part message in MIME format.\n...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [10]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [11]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.